<a href="https://colab.research.google.com/github/Kento-T-JP/OTMM_glycan/blob/main/OTMM_%E6%96%B0%E8%A6%8F%E6%80%A7_local_decimal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 実行について
共有したglycan_data.csvをアップロードしてから実行してください。

「ランタイム(Runtime)→すべてのセルを実行(Run all)」または「Ctrl(command) + F9」で実行してください。

ノートブックの詳細は目次をご覧ください。

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jan 24 08:27:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


# Python version
家や学校のコンピューター（local）の環境はPython 3.10.4。

必要に応じてアップデートする（下のコメントアウトを解除するとアップデートできる）。

結果が大きく変わらなかったので、今回はアップデートを行わない。

In [ ]:
!python --version

Python 3.8.10


In [ ]:
# !sudo apt-get update -y

In [ ]:
# !sudo apt-get install python3.10

In [ ]:
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

In [ ]:
# !python --version

In [ ]:
# pip install pandas==1.5.2

In [ ]:
# pip install numpy==1.23.5

# Glypyのinstall
糖鎖の描画で使用する。

In [ ]:
!pip install glypy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 KB 1.2 MB/s eta 0:00:00


# preprocessor_novelty


In [ ]:
"""OTMMの前処理"""

"""
「新規性」と書いてある部分がこの研究の新規性である。
つまりこの部分を消せば既存の研究になる。
unbound variable(赤い波線)が出るが、処理の流れ的に絶対に定義されるので無視してください
"""

import re
import copy

"""Class Node"""
class Node:
  def __init__(self, no, name, child, child_num):
    # nodesリストのインデックスがorderになっているのでorderは属性に入れない
    self.no = no # position of the text glycan data
    self.name = name # 結合情報を入れる

    self.elder = None # left (immediately elder sibling)
    self.elder_num = None

    self.younger = None # right (immediately younger sibling)
    self.younger_num = None

    self.parent = None
    self.parent_num = None
    
    self.child = [] # down
    self.child.append(child)

    self.child_num = [] # to from OTMM structure
    self.child_num.append(child_num)

  def add_order(self, order): # 必要かわからないので放置
    self.order = order

  def add_elder(self, elder, elder_num):
    self.elder = elder
    self.elder_num = elder_num

  def add_younger(self, younger, younger_num):
    self.younger = younger
    self.younger_num = younger_num

  def add_parent(self, parent, parent_num):
    self.parent = parent
    self.parent_num = parent_num

  def add_child(self, child, child_num):
    self.child.append(child)
    self.child_num.append(child_num)

# タンパク質を消去
def delete_PROT(row):
  if re.fullmatch(r'\(a[0-9]-|\(b[0-9]-$', row["IUPAC Condensed"][-4:]): # [-4:]は文字列の後ろから4文字まで
    row["IUPAC Condensed"] = row["IUPAC Condensed"][:-4] # [:-4]は後ろから4文字を省いたテキスト
  return row

# 糖鎖のテキストデータを行ごとに分ける
def separate_structure(row):
  text = row["IUPAC Condensed"]

  stack = [] # stackを利用
  result = []
  first_flag = False # 最初の単糖を処理したかどうか
  for i, word in enumerate(text):
    if i == 0: # 最初の単糖
      stack.append(0)
      first_flag = True

    if word == '(':
      if first_flag == True: #最初の単糖
        result.append(text[0:i])
        first_flag = False
        stack.append(i)
      else:
        result.append(text[stack.pop():i]) # 単糖
        stack.append(i) # 括弧

    elif word == ')':
      result.append(text[stack.pop():i + 1]) # 括弧
      if text[i+1] != '[':
        stack.append(i + 1) # 単糖

    elif word == '[':
      result.append(text[i]) # 角括弧
      stack.append(i + 1) # 単糖

    elif word == ']':
      result.append(text[i]) # 角括弧
      if text[i+1] != '[':
        stack.append(i + 1) # 単糖
    
    # 途中で括弧などが終わっていてもスタックをすべて使いきれるようにする
    if i == len(text)-1: # "文字数-1"が最後の文字のインデックス
      result.append(text[stack.pop():i + 1])

  row["IUPAC Condensed"] = result
  return row

# 親子関係を抽出（この時点では1人の親が複数の子供を持つ）
def get_structure(result):
  nodes = [] # add nodes of glycan (tree)
  parent_index = set()

  i = len(result)-1
  while i >= 0:
    # ルートのみ例外処理（親がいないので結合情報を入れない）
    if i == len(result)-1:
      if re.fullmatch(r'\([a-z][0-9]-[0-9]\)', result[i-1]) or re.fullmatch(r'\([a-z][0-9]-[0-9]\/[0-9]\)', result[i-1]) or re.fullmatch(r'\([a-z][0-9]-[a-zA-Z]-[0-9]\)', result[i-1]): # 結合様式→子供が1人だけの時
        # print("OK00")
        # print(i)
        nodes.append(Node(i, result[i], result[i-2], i-2)) # nodes[0]==root
        parent_index.add(i)
        # print("root:", result[i-2], i-2, "←", result[i], i, "☆bond:", result[i-1])

      elif result[i-1] == "]": # 分岐の時
        # print("OK01")
        # print(i)
        nodes.append(Node(i, result[i], result[i-3], i-3)) # nodes[0]==root
        parent_index.add(i)
        # print("root:", result[i-3], i-3, "←", result[i], i, "☆bond:", result[i-2])

    # 通常の処理
    elif re.fullmatch(r'[0-9a-zA-Z]+', result[i]) or re.fullmatch(r'[a-zA-z]-[0-9a-zA-Z]+', result[i]) or re.fullmatch(r'[0-9a-zA-Z]+[0-9]\/[0-9][a-zA-Z]', result[i]): # 単糖の場合
      # 1:1の親子関係を認識
      if re.fullmatch(r'\([a-z][0-9]-[0-9]\)', result[i-1]) or re.fullmatch(r'\([a-z][0-9]-[0-9]\/[0-9]\)', result[i-1]) or re.fullmatch(r'\([a-z][0-9]-[a-zA-Z]-[0-9]\)', result[i-1]): # 結合様式
        # print("OK1")
        # print(i)
        nodes.append(Node(i, result[i]+result[i+1], result[i-2], i-2)) # +result[i+1]が新規性
        parent_index.add(i)
        # print(result[i-2], i-2, "←", result[i], i, "☆bond:", result[i-1])
      # 分岐の始まりの場合
      elif result[i-1] == "]":
        parent = result[i] # 分岐元の親を保存（最初の分岐の親）。必ず"["の直後に対応する"]"が来るので1つの変数に適時入れるだけで問題ない
        i_parent = i
        # print(i)
        i, parent_index = branch(result, i-1, nodes, parent_index, parent, i_parent) # iは"["に対応する"]の1つ前のインデックス
        # print("after", i)
        # print("＜branch関数終わり＞")

      elif i == 0: #末尾（葉）の場合
        # print(i)
        nodes.append(Node(i, result[i]+result[i+1], None, None)) # +result[i+1]が新規性
        # 親はいない

    # 分岐の始まりの場合
    elif result[i] == "]":
      if result[i+1] == "[":
        i, parent_index = branch(result, i, nodes, parent_index, parent, i_parent)
      else:
        parent = result[i+1]
        i_parent = i+1
        i, parent_index = branch(result, i, nodes, parent_index, parent, i_parent)
    
    # 分岐の終わりの場合
    elif result[i] == "[":
      # 分岐が連続する場合
      if result[i-1] == "]":
        # 最初の分岐の親がこの分岐の親
        # print("＜新たなbranch＞")
        # print("OK2")
        for node in nodes:
          if node.no == i_parent:
            node.add_child(result[i-3], i-3)
        # print(result[i-3], i-3, "←", parent, i_parent, "☆bond:", result[i-2]) # その他1:1の親子関係については通常の処理で対応可能

      # その階層の分岐が終わる場合（最後の分岐）
      elif re.fullmatch(r'\([a-z][0-9]-[0-9]\)', result[i-1]) or re.fullmatch(r'\([a-z][0-9]-[0-9]\/[0-9]\)', result[i-1]) or re.fullmatch(r'\([a-z][0-9]-[a-zA-Z]-[0-9]\)', result[i-1]): # 結合様式
        # print("OK3")
        # print("＜新たなbranch＞")
        for node in nodes:
          if node.no == i_parent:
            node.add_child(result[i-2], i-2)
            # print(node.no, node.child)
        # print(result[i-2], i-2, "←", parent,i_parent,  "☆bond:", result[i-1])
        # print("＜branch終わり＞\n")
    i -= 1

  # print("\nnumber of nodes:",len(nodes), "\n")
  # for node in nodes:
    # print(node.no, node.name, node.child)
  # print()
  # print(parent_index)

  return nodes

# get_structureで使う関数
def branch(result, j, nodes, parent_index, parent, j_parent):
  # print("\n＜branch始まり＞")
  start = j
  # print(j)
  # print(j_parent)
  while result[j] != "[":
    # 分岐中の親子関係を記述
    if j == start: # ]のとき
      if j+1 in parent_index: # 一つ右のノードが既出の親の場合
        # print(j+1)
        if result[j+1] != result[-1]: # ルートノードではないとき（ルートノードは既に子どもを登録済み）
          for node in nodes:
            if node.no == j+1: # j-2の親が見つかったら
              node.add_child(result[j-2], j-2)
      elif result[j+1] == "[": # 1つ右が"["の場合
          # print(j)
          for node in nodes:
            if node.no == j_parent:
              node.add_child(result[j-2], j-2)
          # print(result[j-2], j-2, "←", parent, j_parent, "☆bond:", result[j-1]) # その他1:1の親子関係については通常の処理で対応可能
      else: # 初めて親が出た時
        # print(j+1)
        nodes.append(Node(j+1, result[j+1]+result[j+2], result[j-2], j-2)) # +result[j+2]が新規性
        parent_index.add(j+1)
        # print(result[j-2], j-2, "←", result[j+1], j+1, "☆bond:", result[j-1])

    # 分岐中の1:1の親子関係を認識
    elif re.fullmatch(r'[0-9a-zA-Z]+', result[j]) or re.fullmatch(r'[a-zA-z]-[0-9a-zA-Z]+', result[j]) or re.fullmatch(r'[0-9a-zA-Z]+[0-9]\/[0-9][a-zA-Z]', result[j]): # 単糖
      if re.fullmatch(r'\([a-z][0-9]-[0-9]\)', result[j-1]) or re.fullmatch(r'\([a-z][0-9]-[0-9]\/[0-9]\)', result[j-1]) or re.fullmatch(r'\([a-z][0-9]-[a-zA-Z]-[0-9]\)', result[j-1]): # 結合様式
        # print(j)
        nodes.append(Node(j, result[j]+result[j+1], result[j-2], j-2)) # +result[j+1]が新規性
        parent_index.add(j)
        # print(result[j-2], j-2, "←", result[j], j, "☆bond:", result[j-1])

    # 分岐の中の分岐に対応
    elif result[j] == "]":
      if result[j+1] == "[": # 3つ以上の分岐のときの2つ目以上の分岐
        parent = copy.deepcopy(parent) # 今の親を維持
        j_parent = copy.deepcopy(j_parent) # # 今の親を維持
      else: # 1つ目の分岐のとき
        parent = result[j+1] # 1つ右が親（参照渡しを利用してglobalに影響を与える）
        j_parent = j+1 # 1つ右が親（参照渡しを利用してglobalに影響を与える）
      # print(j)
      # print("\n＜新たなbranch＞")
      j, parent_index = branch(result, j, nodes, parent_index, parent, j_parent) # 再帰
      # print(j)
      j -= 1 # この処理で再帰後のresult[j]は[になり、次のif文に引っかからなくなる

      # 一つ左が結合様式の時（j_parent_branchの子どもの時）
      if re.fullmatch(r'\([a-z][0-9]-[0-9]\)', result[j-1]) or re.fullmatch(r'\([a-z][0-9]-[0-9]\/[0-9]\)', result[j-1]) or re.fullmatch(r'\([a-z][0-9]-[a-zA-Z]-[0-9]\)', result[j-1]): # 結合様式
        if re.fullmatch(r'[0-9a-zA-Z]+', result[j-2]) or re.fullmatch(r'[a-zA-z]-[0-9a-zA-Z]+', result[j-2]) or re.fullmatch(r'[0-9a-zA-Z]+[0-9]\/[0-9][a-zA-Z]', result[j-2]): # 単糖
          # print(j_parent_branch)
          for node in nodes:
            if node.no == j_parent:
              node.add_child(result[j-2], j-2)

      # 1つ左が"]"のとき（分岐が3つ以上になっているとき or "]["←このようになっているとき）
      if result[j-1] == "]":
        if re.fullmatch(r'\([a-z][0-9]-[0-9]\)', result[j-2]) or re.fullmatch(r'\([a-z][0-9]-[0-9]\/[0-9]\)', result[j-2]) or re.fullmatch(r'\([a-z][0-9]-[a-zA-Z]-[0-9]\)', result[j-2]): # 結合様式
          if re.fullmatch(r'[0-9a-zA-Z]+', result[j-3]) or re.fullmatch(r'[a-zA-z]-[0-9a-zA-Z]+', result[j-3]) or re.fullmatch(r'[0-9a-zA-Z]+[0-9]\/[0-9][a-zA-Z]', result[j-3]): # 単糖
            for node in nodes:
              if node.no == j_parent:
                node.add_child(result[j-3], j-3)
      # print("＜branch関数終わり＞")

    j -= 1 # どんどん前に行く

    # 葉にも対応
    # print(j+1)
    if result[j] == "[":
      # print(j+1)
      nodes.append(Node(j+1, result[j+1]+result[j+2], None, None)) # +result[j+2]が新規性

      if result[j-1] == "]": # 1つ上の階層の親に行かないようにする
        pass

  return j+1, parent_index #"["の1つ前

"""OTMM独自の処理"""
# OTMM用に兄弟関係を抽出（1人の親が1人の子供を持つようにする）
def create_siblings(nodes):
  for node in nodes:
    if len(node.child) >= 2: # 子供が2人以上いるとき
      # print(node.no, node.child)
      parent = node
      for i in range(0,len(parent.child)):
        # print(i)
        if i == 0: # 先頭の子供がi=0（すなわち、eldest（長男）のとき）
          child = parent.child[i] # 子供を入れる
          child_num = parent.child_num[i]
          # print(child_num)
          sibling = parent.child[i+1]
          sibling_num = parent.child_num[i+1]
          # print(sibling_num)
          for node in nodes:
            if node.no == child_num:
              node.add_younger(sibling, sibling_num) # 若い
              node.add_parent(parent.name, parent.no) # 親の処理
            elif node.no == sibling_num:
              node.add_elder(child, child_num) # 年上

        elif i == len(parent.child)-1: # youngestのとき
          youngest = parent.child[i]
          youngest_num = parent.child_num[i]
          #  print(youngest_num)
          elder = parent.child[i-1]
          elder_num = parent.child_num[i-1]
          #  print(elder_num)
          for node in nodes:
            if node.no == youngest_num:
               node.add_elder(elder, elder_num) # 年上のみ

        else:
          elder = parent.child[i]
          elder_num = parent.child_num[i]
          #  print(elder_num)
          younger = parent.child[i+1]
          younger_num = parent.child_num[i+1]
          #  print(younger_num)
          for node in nodes:
            if node.no == elder_num:
               node.add_younger(younger, younger_num) # 若い
            elif node.no == younger_num:
               node.add_elder(elder, elder_num) # 年上
      
      #更新する
      del parent.child[1:len(parent.child)] # 最も近い子供以外を消去
      parent.child = parent.child[0]
      del parent.child_num[1:len(parent.child_num)] # 最も近い子供以外を消去
      parent.child_num = parent.child_num[0]

    elif len(node.child) == 1: # 子供が1人の時
      node.child = node.child[0]
      node.child_num = node.child_num[0]

    elif node.child[0] == None:
      node.child = None
      node.child_num = None

  return nodes

# 親を登録（上記の兄弟の処理が終わっている前提）
def create_parent(nodes):
  for node in nodes:
    if node.child != None:
      parent = node
      child = node.child
      child_num = node.child_num
      for node in nodes:
        if node.no == child_num:
          node.add_parent(parent.name, parent.no)

  return nodes

"""単糖のインスタンスをNodeの属性（parent、child、elder、younger）に代入"""

# 単糖の名前ではなくNodeインスタンスを入れる
def set_instance(nodes):
  for node in nodes:
    # parentをNodeインスタンスに
    if node.parent != None:
      for instance in nodes:
        if instance.no == node.parent_num:
          node.parent = instance # 親のインスタンスを入れる
    # childをNodeインスタンスに
    # if node.child[0] != None: # childのみリスト
    if node.child != None: # childのみリスト
      for instance in nodes:
        # if instance.no == node.child_num[0]:
        if instance.no == node.child_num:
          # node.child[0] = instance # 子のインスタンスを入れる
          node.child = instance
    # youngerをNodeインスタンスに
    if node.younger != None:
      for instance in nodes:
        if instance.no == node.younger_num:
          node.younger = instance # youngerのインスタンスを入れる
    # elderをNodeインスタンスに
    if node.elder != None:
      for instance in nodes:
        if instance.no == node.elder_num:
          node.elder = instance # elderのインスタンスを入れる
  return nodes

def find_mark(row):
  i = 0
  glycan = row["IUPAC Condensed"]
  for node in glycan:
    # if node.child == None and node.elder == None: # 厳密にはyoungerがいるのでOTMMでは末端ではない
    #   node.leaf_order = i
    #   i += 1
    if node.child == None and node.younger == None: # これが厳密な末端の葉
      node.leaf_order = i
      i += 1
  row["IUPAC Condensed"] = glycan
  return row

# algorism_decimal

In [ ]:
"""
主にlikelihoodとlearningで使用するアルゴリズム
新規性のある研究も既存の研究もこの部分は同じ
小数点の加算と減算しか処理が無いのでDecimal(固定小数点)で誤差を無くす
"""

import math
import pandas as pd
import copy # Pythonでは関数においてミュータブルな変数は参照渡しのため
# import gc
from decimal import Decimal

"""
log(x+y)の近似
sum of logs
"""
# math.exp(0) = 1なので条件分岐する必要ない
# log(1+e^(y-x))をすることで、e^(y-x)が非常に小さい場合log(1+e^(y-x))≒0を出力できる→ほぼxが返される
def smoothmax(x, y):
  try:
    return x + Decimal(str(math.log(Decimal(str(1)) + Decimal(str(math.exp(y-x)))))) # exp(±710)くらいでエラーが発生
  except: 
    # logsumexpと同じ処理
    if y-x > 0: # e^(y-x)が非常に大きくなるので1+ e^(y-x)≒e^(y-x)とする
      return copy.deepcopy(y)
    elif y-x < 0:
      return copy.deepcopy(x)

# math.expの指数の範囲を狭めたが特に変化なし
# def smoothmax(x, y):
#   if abs(y-x) <= 20:
#     return x + math.log(1 + math.exp(y-x))
#   else: 
#     # logsumexpと同じ処理
#     if y-x > 0: # e^(y-x)が非常に大きくなるので1+ e^(y-x)≒e^(y-x)とする
#       return y
#     elif y-x < 0:
#       return x

"""Calculate upward and backward"""
def calc_up(q, p, back, a_a, b, state_set):
  #calculate up
  if p.child == None:
    return b.at[q, p.name]
  else:
    total = Decimal(str(0))
    for m in state_set:
      if back.at[m, p.child.no] == 100: # あり得ない値（100）を発見
        print("100(unexpected value) found at up") # 計算していないbackを使っている状態
      # j is the p's eldest children (thus, j == p.child)
      if total == 0 and m == state_set[0]: # 最初の値はそのまま代入
        total += a_a[q][m] + back.at[m, p.child.no] # 対数なので足し算は掛け算
      else:
        total = copy.deepcopy(smoothmax(total, a_a[q][m] + back.at[m, p.child.no])) #totalは和なのでsmoothmaxにtotalを代入すればよい
    return b.at[q, p.name] + total

def calc_back(m, j, up, back, a_b, state_set):
  #calculate back
  # print(back)
  if j.younger == None: # youngest child and root
    if up.at[m, j.no] == 100:
      print("100(unexpected value) found at back") # 計算していないupを使っている状態
    return up.at[m, j.no]
  else:
    total = Decimal(str(0))
    for l in state_set:
      if total == 0 and l == state_set[0]: # 最初の値はそのまま代入
        total += a_b[m][l] + back.at[l, j.younger.no] # 対数なので足し算は掛け算
      else:
        total = copy.deepcopy(smoothmax(total, a_b[m][l] + back.at[l, j.younger.no])) #totalは和なのでsmoothmaxにtotalを代入すればよい
    return up.at[m, j.no] + total

def calc_likelihood(df, pi, a_a, a_b, b, state_set):
  i = 0
  likelihood = []
  for row in df.itertuples():
    # count how many glycans did we use
    if i == 0:
      print("Number of glycans in likelihood", i)
    elif (i+1)%100 == 0:
      print("Number of glycans in likelihood", i+1) # 0からインデックスが始まるため

    glycan = row[2]

    # 識別子で区別する
    sugar_id = []
    for sugar in glycan:
      sugar_id.append(sugar.no)
    sugar_id = sorted(sugar_id)

    # make upward prob
    up = [[Decimal(str(100))] * len(glycan) for i in [Decimal(str(100))] * len(state_set)]
    up = pd.DataFrame(up, index=state_set, columns=sugar_id)
    
    # make backward prob
    back = [[Decimal(str(100))] * len(glycan) for i in [Decimal(str(100))] * len(state_set)]
    back = pd.DataFrame(back, index=state_set, columns=sugar_id)

    for node in reversed(glycan):
      for state in state_set:
        # print(node.no)
        up.at[state, node.no] = copy.deepcopy(calc_up(state, node, back, a_a, b, state_set))
        # print(type(up.at[state, node.no]))
        back.at[state, node.no] = copy.deepcopy(calc_back(state, node, up, back, a_b, state_set))
        # print(type(back.at[state, node.no]))
    # print(up, "\n")
    
    like = Decimal(str(0))
    for l in state_set:
      if like == 0 and l == state_set[0]:
        like += pi[l] + up.at[l, sugar_id[-1]] # 最もインデックスが大きい数（[-1]）は必ずルート
      else:
        like = copy.deepcopy(smoothmax(like, pi[l] + up.at[l, sugar_id[-1]]))
    likelihood.append(like)
    i += 1

    # メモリリーク（対策）
    # del up, back
    # gc.collect()
  
  return likelihood

"""
Learning(EMアルゴリズム)
"""

"""Forward probability"""
def calc_forward(l, j, down, forward, up, a_a, a_b, b, pi, state_set):
  #calculate forward
  if j.parent == None and j.elder == None and j.younger == None: # when j == root
    # if not re.fullmatch(r'[0-9a-zA-Z]+', j.name): # ルートではないときTrue
    #   print(j.name, j.no)
    # return pi[l]
    return Decimal(str(0)) # 論文で言及されていない部分（とりあえず確率は1でいいらしい）, log(1) = 0
  elif j.elder == None: # eldest children
    total = Decimal(str(0))
    for q in state_set:
      if down.at[q, j.parent.no] == 100:
        print("100(unexpected value) found at forward!")
      if total == 0 and q == state_set[0]:
        total += a_a[q][l] + down.at[q, j.parent.no] + b.at[q, j.parent.name]
      else:
        total = copy.deepcopy(smoothmax(total, a_a[q][l] + down.at[q, j.parent.no] + b.at[q, j.parent.name]))
    return total
  else:
    total = Decimal(str(0))
    for m in state_set:
      if forward.at[m, j.elder.no] == 100:
        print("100(unexpected value) found at forward!!")
      if total == 0 and m == state_set[0]:
        total += a_b[m][l] + forward.at[m, j.elder.no] + up.at[m, j.elder.no] # 対数なので足し算は掛け算
      else:
        total = copy.deepcopy(smoothmax(total, a_b[m][l] + forward.at[m, j.elder.no] + up.at[m, j.elder.no]))
    return total

"""Downward probability"""
def calc_down(l, j, forward, back, pi, a_b, state_set):
  #calculate downward
  if j.parent == None and j.elder == None and j.younger == None: # when j == root
    return pi[l]
  elif j.younger == None:
    if forward.at[l, j.no] == 100:
      print("100(unexpected value) found at down!")
    return forward.at[l, j.no]
  else:
    total = Decimal(str(0))
    for m in state_set:
      if forward.at[l, j.no] == 100:
        print("100(unexpected value) found at down!!")
      if total == 0 and m == state_set[0]:
        total += a_b[l][m] + back.at[m, j.younger.no] # 対数なので足し算は掛け算
      else:
        total = copy.deepcopy(smoothmax(total, a_b[l][m] + back.at[m, j.younger.no]))
    return forward.at[l, j.no] + total

"""Learning（EMアルゴリズム）"""
def EM(df, pi_original, a_a_original, a_b_original, b_original, state_set, label_set, L, epsilon):
  t = 0  
  L_all = [] # 全体の期待値
  print("likelihood before learning", L, "\n")
  L_all.append(L)

  # 参照渡しのため値を別のアドレス（変数）にコピー
  pi = copy.deepcopy(pi_original)
  a_a = copy.deepcopy(a_a_original)
  a_b = copy.deepcopy(a_b_original)
  b = copy.deepcopy(b_original)

  while True:
    print("t(epoc)=", t)
    # L_T = [] # record all likelihoods of T_u

    # 5 of pseudo code
    # initialize mu of T_u
    mu_aa_t = [[Decimal(str(0))] * len(state_set) for i in [Decimal(str(0))] * len(state_set)]

    mu_ab_t = [[Decimal(str(0))] * len(state_set) for i in [Decimal(str(0))] * len(state_set)]

    B_t = [[Decimal(str(0))] * len(label_set) for i in [Decimal(str(0))] * len(state_set)]
    mu_b_t = pd.DataFrame(B_t, index=state_set, columns=label_set)
        
    mu_pi_t = [Decimal(str(0))] * len(state_set)

    # 6 of pseudo code
    count = 0
    for row in df.itertuples():
      if count == 0:
        print("Number of glycans in learning", count)
      elif (count+1)%100 == 0:
        print("Number of glycans in learning", count+1) # 0からインデックスが始まるため

      # 7 of pseudo code
      glycan = row[2]

      # initialize mu of the glycan
      mu_aa_u = [[Decimal(str(0))] * len(state_set) for i in [Decimal(str(0))] * len(state_set)]

      mu_ab_u = [[Decimal(str(0))] * len(state_set) for i in [Decimal(str(0))] * len(state_set)]

      B_u = [[Decimal(str(0))] * len(label_set) for i in [Decimal(str(0))] * len(state_set)]
      mu_b_u = pd.DataFrame(B_u, index=state_set, columns=label_set)
        
      mu_pi_u = [Decimal(str(0))] * len(state_set)

      # 識別子で区別する
      sugar_id = []
      for sugar in glycan:
        sugar_id.append(sugar.no)
      sugar_id = sorted(sugar_id)
      # make upward prob
      up = [[Decimal(str(100))] * len(glycan) for i in [Decimal(str(100))] * len(state_set)]
      up = pd.DataFrame(up, index=state_set, columns=sugar_id)
      
      # make backward prob
      back = [[Decimal(str(100))] * len(glycan) for i in [Decimal(str(100))] * len(state_set)]
      back = pd.DataFrame(back, index=state_set, columns=sugar_id)

      # make forward prob
      forward = [[Decimal(str(100))] * len(glycan) for i in [Decimal(str(100))] * len(state_set)]
      forward = pd.DataFrame(forward, index=state_set, columns=sugar_id)

      # make downward prob
      down = [[Decimal(str(100))] * len(glycan) for i in [Decimal(str(100))] * len(state_set)]
      down = pd.DataFrame(down, index=state_set, columns=sugar_id)

      # U, Bを計算する(7 to 9 of pseudo code)
      # reversed(glycan)...bottom-up and right-to-left dynamic programming procedure
      for node in reversed(glycan):
        for state in state_set:
          # print(node.no)
          up.at[state, node.no] = copy.deepcopy(calc_up(state, node, back, a_a, b, state_set))
          back.at[state, node.no] = copy.deepcopy(calc_back(state, node, up, back, a_b, state_set))

      # D, Fを計算する(10 to 12 of pseudo code)
      # glycan...top-down and left-to-right dynamic programming procedure
      for node in glycan:
        for state in state_set:
          # print(node.no)
          forward.at[state, node.no] = copy.deepcopy(calc_forward(state, node, down, forward, up, a_a, a_b, b, pi, state_set))
          down.at[state, node.no] = copy.deepcopy(calc_down(state, node, forward, back, pi, a_b, state_set))

      # calculate L(T_u), 尤度
      L_u = Decimal(str(0)) # 入力データ1つ1つの尤度を保存
      # L_u2 = 0
      i = glycan[0].no # optional
      # print(i)
      for l in state_set:
        if L_u == 0 and l == state_set[0]:
          L_u += up.at[l, i] + down.at[l, i]
        else:
          L_u = copy.deepcopy(smoothmax(L_u, up.at[l, i] + down.at[l, i]))
        # if L_u2 == 0:
        #   L_u2 = pi_em[l] + up_em.at[l, sugar_id[-1]]
        # else:
        #   L_u2 = smoothmax(L_u2, pi_em[l] + up_em.at[l, sugar_id[-1]])
      # print(L_u, L_u2) # L_u = L_u2だった
      # L_T.append(L_u)

      # 期待値の計算（13 of pseudo code）
      # calculate mu_aa_u[q][l]
      for q in state_set:
        for l in state_set:
          exist_child = False # 該当するchildがいるかどうか
          total = Decimal(str(0))
          for p in glycan:
            if p.child != None:
              exist_child = True
              # p.child.no means j in the thesis of OTMM
              if total == 0:
                total += down.at[q, p.no] + b.at[q, p.name] + a_a[q][l] + back.at[l, p.child.no]
              else:
                total = copy.deepcopy(smoothmax(total, down.at[q, p.no] + b.at[q, p.name] + a_a[q][l] + back.at[l, p.child.no]))
          if exist_child == True or total != 0: # 本来total×L_uなのでtotal=0の時は0（対数変換によって掛け算が足し算になっている）
            mu_aa_u[q][l] = total - L_u # 対数の引き算は割り算
          else:
            mu_aa_u[q][l] = Decimal(str(0)) # 0にしてパラメータの更新に影響が出ないようにする
          # print(total, L_u)
      
      # calculate mu_ab_u[q][l]
      for q in state_set:
        for l in state_set:
          exist_younger = False # younger siblingがいるかどうか
          total = Decimal(str(0))
          for j in glycan:
            if j.younger != None: # X(j) != empty set
              exist_younger = True
              # p.child.no means j in the thesis of OTMM
              if total == 0:
                total += forward.at[q, j.no] + a_b[q][l] + back.at[l, j.younger.no] + up.at[q, j.no]
              else:
                total = copy.deepcopy(smoothmax(total, forward.at[q, j.no] + a_b[q][l] + back.at[l, j.younger.no] + up.at[q, j.no]))
          if exist_younger == True or total != 0:
            mu_ab_u[q][l] = total - L_u # 対数の引き算は割り算
          else:
            mu_ab_u[q][l] = Decimal(str(0))

      # calculate mu_b_u[m][o_h]
      for m in state_set:
        for o_h in label_set:
          exist_oh = False # whether the sugar(o_h) exists or not
          total = Decimal(str(0))
          for i in glycan:
            if i.name == o_h:
              exist_oh = True
              if total == 0:
                total += down.at[m, i.no] + up.at[m, i.no]
              else:
                total = copy.deepcopy(smoothmax(total, down.at[m, i.no] + up.at[m, i.no]))
          if exist_oh == True or total != 0:
            mu_b_u.at[m, o_h] = total - L_u # 対数の引き算は割り算
          else:
            mu_b_u.at[m, o_h] = Decimal(str(0)) # 入力データには単糖o_hが存在しないので期待値は0

      # calculate  mu_pi_u[m]
      for m in state_set:
        mu_pi_u[m] = (pi[m] + up.at[m, sugar_id[-1]]) - L_u # 最も大きい識別子がルートノード

      # for each param, do mu_t = mu_t + mu_u (14 of pseudo code)
      # mu_aa_t = mu_aa_t + mu_aa_u
      for q in state_set:
        for l in state_set:
          if mu_aa_u[q][l] == 0:
            mu_aa_t[q][l] += Decimal(str(0))
          else:
            if mu_aa_t[q][l] == 0:
              mu_aa_t[q][l] += copy.deepcopy(mu_aa_u[q][l])
            else:
              mu_aa_t[q][l] = copy.deepcopy(smoothmax(mu_aa_t[q][l], mu_aa_u[q][l]))

      # mu_ab_t = mu_ab_t + mu_ab_u
      for q in state_set:
        for l in state_set:
          if mu_ab_u[q][l] == 0:
            mu_ab_t[q][l] += Decimal(str(0))
          else:
            if  mu_ab_t[q][l] == 0:
              mu_ab_t[q][l] += copy.deepcopy(mu_ab_u[q][l])
            else:
              mu_ab_t[q][l] = copy.deepcopy(smoothmax(mu_ab_t[q][l], mu_ab_u[q][l]))

      # mu_b_t = mu_b_t + mu_b_u
      for m in state_set:
        for o_h in label_set:
          if mu_b_u.at[m, o_h] == 0:
            mu_b_t.at[m, o_h] += Decimal(str(0))
          else:
            if mu_b_t.at[m, o_h] == 0:
              mu_b_t.at[m, o_h] += copy.deepcopy(mu_b_u.at[m, o_h])
            else:
              mu_b_t.at[m, o_h] = copy.deepcopy(smoothmax(mu_b_t.at[m, o_h],  mu_b_u.at[m, o_h]))

      # mu_pi_t = mu_pi_t + mu_pi_u
      for m in state_set:
        if mu_pi_u[m] == 0:
          mu_pi_t[m] += Decimal(str(0))
        else:
          if mu_pi_t[m] == 0:
            mu_pi_t[m] += copy.deepcopy(mu_pi_u[m])
          else:
            mu_pi_t[m] = copy.deepcopy(smoothmax(mu_pi_t[m], mu_pi_u[m]))
      
      # メモリの開放（メモリリーク対策）
      # del up, back, forward, down
      # del mu_pi_u, mu_aa_u, mu_ab_u, mu_b_u
      # gc.collect()
      count += 1

    # update a_a
    # print(mu_aa_t)
    for q in state_set:
      denominator = Decimal(str(0))
      for l_dash in state_set:
        if denominator == 0 and l_dash == state_set[0]:
          denominator += copy.deepcopy(mu_aa_t[q][l_dash])
        else:
          denominator = copy.deepcopy(smoothmax(denominator, mu_aa_t[q][l_dash]))
      for l in state_set:
        numerator = copy.deepcopy(mu_aa_t[q][l])
        a_a[q][l] = numerator - denominator # 対数の引き算は割り算

    # update a_b
    # print(mu_ab_t)
    for q in state_set:
      denominator = Decimal(str(0))
      for l_dash in state_set:
        if denominator == 0 and l_dash == state_set[0]:
          denominator += copy.deepcopy(mu_ab_t[q][l_dash])
        else:
          denominator = copy.deepcopy(smoothmax(denominator, mu_ab_t[q][l_dash]))
      for l in state_set:
        numerator = copy.deepcopy(mu_ab_t[q][l])
        a_b[q][l] = numerator - denominator

    # update b
    for m in state_set:
      denominator = Decimal(str(0))
      for o_i in label_set:
        if denominator == 0 and o_i == label_set[0]:
          denominator += copy.deepcopy(mu_b_t.at[m, o_i])
        else:
          denominator = copy.deepcopy(smoothmax(denominator, mu_b_t.at[m, o_i]))
      for o_h in label_set:
        numerator = copy.deepcopy(mu_b_t.at[m, o_h])
        # print(numerator, denominator)
        b.at[m, o_h] = numerator - denominator

    # update pi
    for m in state_set:
      numerator = copy.deepcopy(mu_pi_t[m])
      denominator = Decimal(str(0))
      for k in state_set:
        if denominator == 0 and k == state_set[0]:
          denominator += copy.deepcopy(mu_pi_t[k])
        else:
          denominator = copy.deepcopy(smoothmax(denominator, mu_pi_t[k]))
      pi[m] = numerator - denominator

    t += 1
    # calculate L_t(T) using the param updated (17 of the pseudo code)
    likelihoods = calc_likelihood(df, pi, a_a, a_b, b, state_set)
    # print(a_a_em[0])
    # print(a_b_em[0])
    # print(pi_em)
    # print(b_em)
    # print(likelihoods)
    L_all.append(sum(likelihoods)) # 対数（likelihood）の足し算は掛け算
    
    # L_all.append(sum(L_T)) # 現在のパラメータを使って計算をしたいので
    print("Likelihood", L_all[t])
    print("Difference of the likelihoods", L_all[t] - L_all[t-1], "\n")
    if abs(L_all[t] - L_all[t-1]) < epsilon: # or t == 5: # 止まらなかったら困るのでとりあえずt==5で止める
      return pi, a_a, a_b, b, L_all

# parsing

In [ ]:
"""
Parsing (Retrieve what was learned by retrieving the most likely state paths)
Viterbiアルゴリズムで経路探索
"""

import numpy as np
import pandas as pd

# 識別子で区別する
def parse_glycan(glycan, state_set, pi, a_a, a_b, b):
    sugar_id = []
    for sugar in glycan:
        sugar_id.append(sugar.no)
        sugar_id = sorted(sugar_id)

    """Make data variable for phi"""
    phi_up = np.zeros((len(state_set), len(glycan)))
    phi_up = pd.DataFrame(phi_up, index=state_set, columns=sugar_id)

    phi_back = np.zeros((len(state_set), len(glycan)))
    phi_back = pd.DataFrame(phi_back, index=state_set, columns=sugar_id)

    """Calculate phi"""
    # bottom-up and right-to-left dynamic programming procedure
    for node in reversed(glycan):
        for state in state_set:
            # print(node.no)
            phi_up.at[state, node.no] = calc_phi_up(state, node, phi_back, a_a, b, state_set)
            phi_back.at[state, node.no] = calc_phi_back(state, node, phi_up, phi_back, a_b, state_set)

    p = most_likely_prob(pi, phi_up, state_set, sugar_id)
    print("The probabilitythat all labels are outputted along the most likely state transition:", p)
    z = most_likely_state(glycan, state_set, sugar_id, pi, a_a, a_b, b, phi_up, phi_back)
    for node in glycan:
        print("Node:", node.name, node.no,"Most likely state:", z[node.no])
    print()

"""
Calculate psi
top-down and left-to-right manner
"""
def calc_phi_up(q, p, phi_back, a_a, b, state_set):
  #calculate phi up
  if p.child == None:
    return b.at[q, p.name]
  else:
    prob = []
    for m in state_set:
      if phi_back.at[m, p.child.no] == 0:
        print("Unexpected value at calc phi up") # 計算していないphi_backを使っている状態
      # Σの部分をmaxに
      # j is the p's eldest children (thus, j == p.child)
      prob.append(a_a[q][m] + phi_back.at[m, p.child.no]) # 対数なので足し算は掛け算
    return b.at[q, p.name] + max(prob)

def calc_phi_back(m, j, phi_up, phi_back, a_b, state_set):
  #calculate phi back
  if j.younger == None: # youngest
    if phi_up.at[m, j.no] == 0:
      print("Unexpected value at calc phi back") # 計算していないphi_upを使っている状態
    return phi_up.at[m, j.no]
  else:
    prob = []
    for l in state_set:
      # Σの部分をmaxに
      prob.append(a_b[m][l] + phi_back.at[l, j.younger.no]) # 対数なので足し算は掛け算
    return phi_up.at[m, j.no] + max(prob)

# returns the most likely state for the given node
def psi_up(q, p, phi_back, a_a, state_set):
  if p.child == None: # ψ_upにおいてpは必ず子供を持つのでこの条件は該当しないはず
    print("ERROR! node p shoud have a child!")
  else:
    prob = np.empty(0)
    for m in state_set: # m proceeds from 0(state_set[0]) to state_set[-1]
      if phi_back.at[m, p.child.no] == 0:
        print("Unexpected value at psi up") # 計算していないbackを使っている状態
      # Σの部分をargmaxに
      # j is the p's eldest children (thus, j == p.child)
      # print(q, m, p.child.no)
      prob = np.insert(prob, m, a_a[q][m] + phi_back.at[m, p.child.no]) # 対数なので足し算は掛け算
    return np.argmax(prob) # m represents a state and return it

# returns the most likely state for the given node
def psi_back(m, j, phi_back, a_b, state_set):
  if j.younger == None: # ψ_backにおいてjは必ずyoungerを持つのでこの条件は該当しないはず
    print("ERROR! j shoud have a younger sibling!")
  else:
    prob = np.empty(0)
    for l in state_set: # l proceeds from 0(state_set[0]) to state_set[-1]
      # Σの部分をargmaxに
      prob = np.insert(prob, l, a_b[m][l] + phi_back.at[l, j.younger.no]) # 対数なので足し算は掛け算
    return np.argmax(prob) # l represents a state and return it

def most_likely_prob(pi, phi_up, state_set, sugar_id):
  likely = []
  for l in state_set:
    likely.append(pi[l] + phi_up.at[l, sugar_id[-1]])

  most_likely = max(likely)
  return most_likely

"""Calculate the most likely state for the given node"""
def most_likely_state(glycan, state_set, sugar_id, pi, a_a, a_b, b, phi_up, phi_back):
  z = dict()
  # top-down and left-to-right manner
  for node in glycan:
    # print(node.no)
    if node.no == sugar_id[-1]: # root
      prob = np.empty(0)
      for l in state_set:
        prob = np.insert(prob, l, pi[l] + phi_up.at[l, node.no]) # node.no==sugar_id[-1]
      z[node.no] = np.argmax(prob) # return the most likely state of the root node
    elif node.elder == None and node.parent != None:
      z[node.no] = psi_up(z[node.parent.no], node.parent, phi_back, a_a, state_set)
    else:
      z[node.no] = psi_back(z[node.elder.no], node.elder, phi_back, a_b, state_set)
  return z

# main(otmm_decimal_novelty)

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd
import sys
# import gc
import time
import os
import csv
from decimal import Decimal

# import preprocessor_novelty as pp
# # import algorism as alg
# import algorism_decimal as alg
# # import algorism_decimal_all as alg
# # import algorism_round as alg
# # import algorism_int as alg
# import parsing
# # import parsing_decimal as parsing

"""Class Node"""
class Node:
  def __init__(self, no, name, child, child_num):
    # nodesリストのインデックスがorderになっているのでorderは属性に入れない
    self.no = no # position of the text glycan data
    self.name = name # 結合情報を入れる

    self.elder = None # left (immediately elder sibling)
    self.elder_num = None

    self.younger = None # right (immediately younger sibling)
    self.younger_num = None

    self.parent = None
    self.parent_num = None
    
    self.child = [] # down
    self.child.append(child)

    self.child_num = [] # to from OTMM structure
    self.child_num.append(child_num)

  def add_order(self, order): # 必要かわからないので放置
    self.order = order

  def add_elder(self, elder, elder_num):
    self.elder = elder
    self.elder_num = elder_num

  def add_younger(self, younger, younger_num):
    self.younger = younger
    self.younger_num = younger_num

  def add_parent(self, parent, parent_num):
    self.parent = parent
    self.parent_num = parent_num

  def add_child(self, child, child_num):
    self.child.append(child)
    self.child_num.append(child_num)

"""前処理を行う関数"""
def make_OTMM(row):
  glycan = row["IUPAC Condensed"]
  glycan = get_structure(glycan)
  glycan = create_siblings(glycan)
  glycan = create_parent(glycan)
  glycan = set_instance(glycan)

  row["IUPAC Condensed"] = glycan

  return row

"""データの表示"""
def print_OTMM(glycan):
  print("This is a glycan for OTMM\n")
  for i in range(len(glycan)):
    print("ID:", glycan[i].no)
    print("name:", glycan[i].name)

    if glycan[i].parent == None:
      print("parent:", glycan[i].parent)
    else:
      print("parent:", glycan[i].parent.name, glycan[i].parent.no)

    if glycan[i].child == None:
      print("child:", glycan[i].child)
    else:
      print("child:", glycan[i].child.name, glycan[i].child.no)

    if glycan[i].elder == None:
      print("elder sibling:", glycan[i].elder)
    else:
      print("elder sibling:", glycan[i].elder.name, glycan[i].elder.no)
    
    if glycan[i].younger == None:
      print("younger sibling:", glycan[i].younger)
    else:
      print("younger sibling:", glycan[i].younger.name, glycan[i].younger.no)
    print()

def main(argv):
  num_data = input('Number of data. Input the number or "max":')
  epsilon = input("Epsilon:")
  n = input("Number of states:")

  """データの前処理"""
  # データをdfに読み込み
  # 必ずGoogleColabにglycan_data.csvをアップロードすること
  df = pd.read_csv("glycan_data.csv") # 木下研から頂いたデータ

  # データが無い行を消去
  df = df.dropna(subset=['IUPAC Condensed'])

  drop_index = [] #消去する糖鎖のindexを格納する

  # ?が入っている糖鎖は構造が不明な糖鎖
  for row in df.itertuples(): # itertuples()で行ごとにタプルで取り出す
    if "?" in row[2]:
      drop_index.append(row[0])

  # ?が含まれる糖鎖構造を消去
  df = df.drop(drop_index)

  # indexを振りなおす
  df = df.reset_index(drop=True)

  #1行ごとにdelete_PROT()を呼び出す
  df = df.apply(delete_PROT, axis=1) # axis=1とすることで1行ずつの処理になる

  #1行ごとにseparate_structure()を呼び出す
  df = df.apply(separate_structure, axis=1) # axis=1とすることで1行ずつの処理になる

  """
  データ量の制限
  """
  if num_data == "max":
    df = df # データ全部
  else:
    df = df.head(int(num_data))
  print("Number of data:", len(df))

  # プログラム的にIUPAC Condensedが2番目に来ること！（row[2]と記述している部分があるから）
  df = df.drop(['Motifs', 'Subsumption Level', 'ChEBI', 'Monoisotopic Mass', 'Species'], axis=1)

  df = df.apply(make_OTMM, axis=1)

  df = df.apply(find_mark, axis=1)

  """アルゴリズム"""
  label_set = set()

  for row in df.itertuples():
    glycan = row[2]
    for node in glycan:
      label_set.add(node.name)

  # pandas1.5以上はlistに
  label_set = list(label_set)
  # local環境ではソートしないと実行するごとに順番が変わってしまう
  # Google colabではset型でも順番は一定
  label_set = sorted(label_set)

  # print(label_set)

  print("Number of labels:", len(label_set))
  # print(label_set)

  n = int(n)
  state_set = []
  for i in range(n):
    state_set.append(i)

  # pandas1.5以上はlistに
  state_set = list(state_set)

  print("Number of states:", len(state_set))
  # print(state_set)

  """パラメータのinitialize"""
  """初期状態確率分布 π"""
  np.random.seed(seed=0)
  pi = np.random.rand(len(state_set))
  pi = pi/pi.sum()
  """対数変換"""
  pi = np.log(pi)
  """Decimal型に"""
  pi = pi.tolist()
  for i in range(len(pi)):
    pi[i] = Decimal(str(float(pi[i])))

  """状態遷移確率 A
  parent - node(me) ・・・ A_a
  """
  np.random.seed(seed=1)
  a_a = np.random.rand(len(state_set), len(state_set))
  a_a = a_a/a_a.sum(axis=1).reshape(-1, 1)
  """対数変換"""
  a_a = np.log(a_a)
  a_a = a_a.tolist()
  for i in range(len(a_a)):
    for j in range(len(a_a[i])):
      a_a[i][j] = Decimal(str(float(a_a[i][j])))

  """elder - node(me) ・・・A_b"""
  np.random.seed(seed=2)
  a_b = np.random.rand(len(state_set), len(state_set))
  a_b = a_b/a_b.sum(axis=1).reshape(-1, 1)
  """対数変換"""
  a_b = np.log(a_b)
  a_b = a_b.tolist()
  for i in range(len(a_b)):
    for j in range(len(a_b[i])):
      a_b[i][j] = Decimal(str(float(a_b[i][j])))

  """ラベル出力確率分布 B"""
  np.random.seed(seed=3)
  matrix_B = np.random.rand(len(state_set), len(label_set))
  matrix_B = matrix_B/matrix_B.sum(axis=1).reshape(-1, 1)
  """対数変換"""
  matrix_B = np.log(matrix_B)
  matrix_B = matrix_B.tolist()
  for i in range(len(matrix_B)):
    for j in range(len(matrix_B[i])):
      matrix_B[i][j] = Decimal(str(float(matrix_B[i][j])))
  b = pd.DataFrame(matrix_B, index=state_set, columns=label_set)

  """初期の尤度"""
  print("\nStart time.perf_counter()")
  start = time.perf_counter() # time.time()よりtime.perf_counter()の方が精度が高い
  likelihood = calc_likelihood(df, pi, a_a, a_b, b, state_set)
  L = sum(likelihood) # π（全てを掛け合わせる）なのでsumでよい

  """しきい値"""
  epsilon = float(epsilon) # とりあえずこのくらい

  # gc.collect() # メモリの開放（メモリリーク対策）

  print("\nLearning")
  new_pi, new_a_a, new_a_b, new_b, L_all = EM(df, pi, a_a, a_b, b, state_set, label_set, L, epsilon)
  end = time.perf_counter()
  print()

  print("\nEnd time.perf_counter()\n")
  print("pi updated\n", new_pi)
  print("α updated\n", new_a_a)
  print("β updated\n", new_a_b)
  print("b updated\n", new_b)

  print("\nThe processing time in learning:", end-start, "seconds")

  # 結果を格納するディレクトリを作成
  dir_path =  'result'+'_'+'novelty'+'_'+str(len(df))+'_'+str(epsilon)+'_'+str(len(state_set))+'_'+str(len(label_set))
  os.makedirs(dir_path, exist_ok=True)
  os.chdir(dir_path)  # 相対パス

  # output (新規性)
  time_name = 'time'+'_'+str(len(df))+'_'+str(epsilon)+'_'+str(len(state_set))+'_'+str(len(label_set))+'.txt'
  with open(time_name, 'w') as f:
    f.write("Time in learning: "+str(end-start)+ " seconds")

  pi_name = 'pi'+'_'+str(len(df))+'_'+str(epsilon)+'_'+str(len(state_set))+'.csv'
  np.savetxt(pi_name, new_pi, delimiter=',')
  a_a_name = 'a_a'+'_'+str(len(df))+'_'+str(epsilon)+'_'+str(len(state_set))+'_'+str(len(state_set))+'.csv'
  np.savetxt(a_a_name, new_a_a, delimiter=',')
  a_b_name = 'a_b'+'_'+str(len(df))+'_'+str(epsilon)+'_'+str(len(state_set))+'_'+str(len(state_set))+'.csv'
  np.savetxt(a_b_name, new_a_b, delimiter=',')
  b_name = 'b'+'_'+str(len(df))+'_'+str(epsilon)+'_'+str(len(state_set))+'_'+str(len(label_set))+'.csv'
  new_b.to_csv(b_name)

  # Likelihoodを出力
  likelihood_name = 'likelihood'+'_'+str(len(df))+'_'+str(epsilon)+'_'+str(len(state_set))+'_'+str(len(label_set))+'.csv'
  with open(likelihood_name, 'wt', encoding='utf-8') as f:
    # ライター（書き込み者）を作成
    writer = csv.writer(f)
    # ライターでデータ（リスト）をファイルに出力
    writer.writerow(L_all)

  os.chdir("..")
  """
  Parsing
  viterbiアルゴリズムで経路探索
  """
  print("\nParsing")
  # 解析する糖鎖を1つ選ぶ（今回はcsvファイルの先頭の糖鎖）
  glycan = df["IUPAC Condensed"][0] # glycan[0]
  # ParsingにDecimalが扱えない処理があるのでfloatに変換
  new_pi = np.float_(new_pi)
  new_a_a = np.float_(new_a_a)
  new_a_b = np.float_(new_a_b)
  new_b = new_b.astype('float64')
  parse_glycan(glycan, state_set, new_pi, new_a_a, new_a_b, new_b)

  return 0

# 2回目からはここから実行してください
使用するデータの数、しきい値、状態数は自分で設定してください（実行すると聞かれます）。

※データ数を10くらいにすると、すぐに結果が見れます。

※maxと入力するとmaxは5195のデータが使われます。

In [ ]:
path = os.getcwd()
if path == "/content":
  if __name__ == '__main__':
    sys.exit(main(sys.argv))
elif path == "/":
  os.chdir("content")
  if __name__ == '__main__':
    sys.exit(main(sys.argv))
else:
  os.chdir("..")
  if __name__ == '__main__':
    sys.exit(main(sys.argv))

KeyboardInterrupt: ignored

Google colab無料版の場合、データ数が5195だと1エポック当たり約14.8分かかる。

## MainでParsingした糖鎖構造

In [ ]:
from glypy.io import iupac
from glypy import plot

print("GlcNAc(b1-2)Man(a1-3)[GlcNAc(b1-4)][Gal(b1-4)GlcNAc(b1-2)Man(a1-6)]Man(b1-4)GlcNAc(b1-4)[D-Fuc(a1-6)]GlcNAc")
glycan_glypy = iupac.loads("GlcNAc(b1-2)Man(a1-3)[GlcNAc(b1-4)][Gal(b1-4)GlcNAc(b1-2)Man(a1-6)]Man(b1-4)GlcNAc(b1-4)[Fuc(a1-6)]GlcNAc", dialect="simple")
dt, ax = plot.plot(glycan_glypy, label=True)
ax.figure.set_figwidth(8)
ax.figure.set_figheight(4)
lo, hi = ax.get_ylim()
ax.set_ylim(lo / 2, hi / 1.2)

# Parsing

In [ ]:
import pandas as pd
glycans = [["High mannose", "Man(a1-2)Man(a1-2)Man(a1-2)Man(a1-3)[Man(a1-2)Man(a1-2)Man(a1-3)[Man(a1-2)Man(a1-2)Man(a1-2)Man(a1-6)]Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc(b1-"],
           ["Complex", "Neu5Ac(a2-6)Gal(b1-4)GlcNAc(b1-2)[Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-6)]GlcNAc(b1-4)]Man(a1-3)[Neu5Ac(a2-6)Gal(b1-4)[Neu5Ac(a2-6)]GlcNAc(b1-2)Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc(b1-"],
           ["Hybrid", "Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-2)Man(a1-3)[Man(a1-3)[Man(a1-6)]Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc(b1-"]]
parsing_glycan = pd.DataFrame(glycans, columns=["type", "IUPAC Condensed"])

parsing_glycan

In [ ]:
#1行ごとにdelete_PROT()を呼び出す
parsing_glycan = parsing_glycan.apply(delete_PROT, axis=1) # axis=1とすることで1行ずつの処理になる

#1行ごとにseparate_structure()を呼び出す
parsing_glycan = parsing_glycan.apply(separate_structure, axis=1) # axis=1とすることで1行ずつの処理になる

parsing_glycan = parsing_glycan.apply(make_OTMM, axis=1)

parsing_glycan = parsing_glycan.apply(find_mark, axis=1)

parsing_glycan

In [ ]:
path = os.getcwd()
if path == "/content":
  pass
elif path == "/":
  os.chdir("content")
else:
  os.chdir("..")

In [ ]:
data = input("Number of data?")
epsilon = input("Threshold?")
state = input("Number of states?")
label = input("Number of labels?")

dir_path =  'result'+'_'+'novelty'+'_'+data+'_'+epsilon+'_'+state+'_'+label
os.chdir(dir_path)

# データの読み取り
pi_name = 'pi'+'_'+data+'_'+epsilon+'_'+state+'.csv'
new_pi = np.loadtxt(pi_name, delimiter=",", dtype = "unicode")

a_a_name = 'a_a'+'_'+data+'_'+epsilon+'_'+state+'_'+state+'.csv'
new_a_a = np.loadtxt(a_a_name, delimiter=",", dtype = "unicode")

a_b_name = 'a_b'+'_'+data+'_'+epsilon+'_'+state+'_'+state+'.csv'
new_a_b = np.loadtxt(a_b_name, delimiter=",", dtype = "unicode")

b_name = 'b'+'_'+data+'_'+epsilon+'_'+state+'_'+label+'.csv'
new_b = pd.read_csv(b_name, header=0, index_col=0, encoding="SHIFT-JIS")

In [ ]:
new_pi = np.float_(new_pi)
new_a_a = np.float_(new_a_a)
new_a_b = np.float_(new_a_b)
new_b = new_b.astype('float64')

In [ ]:
n = int(state)
state_set = []
for i in range(n):
  state_set.append(i)
print(state_set)

In [ ]:
glycan = parsing_glycan["IUPAC Condensed"][0]
parse_glycan(glycan, state_set, new_pi, new_a_a, new_a_b, new_b)

In [ ]:
from glypy.io import iupac
from glypy import plot

print(parsing_glycan["type"][0])
print("Man(a1-2)Man(a1-2)Man(a1-2)Man(a1-3)[Man(a1-2)Man(a1-2)Man(a1-3)[Man(a1-2)Man(a1-2)Man(a1-2)Man(a1-6)]Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc")
glycan_glypy = iupac.loads("Man(a1-2)Man(a1-2)Man(a1-2)Man(a1-3)[Man(a1-2)Man(a1-2)Man(a1-3)[Man(a1-2)Man(a1-2)Man(a1-2)Man(a1-6)]Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc", dialect="simple")
dt, ax = plot.plot(glycan_glypy, label=True)
ax.figure.set_figwidth(8)
ax.figure.set_figheight(4)
lo, hi = ax.get_ylim()
ax.set_ylim(lo / 2, hi / 1.2)

In [ ]:
glycan = parsing_glycan["IUPAC Condensed"][1]
parse_glycan(glycan, state_set, new_pi, new_a_a, new_a_b, new_b)

In [ ]:
print(parsing_glycan["type"][1])
print("Neu5Ac(a2-6)Gal(b1-4)GlcNAc(b1-2)[Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-6)]GlcNAc(b1-4)]Man(a1-3)[Neu5Ac(a2-6)Gal(b1-4)[Neu5Ac(a2-6)]GlcNAc(b1-2)Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc")
glycan_glypy = iupac.loads("Neu5Ac(a2-6)Gal(b1-4)GlcNAc(b1-2)[Neu5Ac(a2-3)Gal(b1-3)[Neu5Ac(a2-6)]GlcNAc(b1-4)]Man(a1-3)[Neu5Ac(a2-6)Gal(b1-4)[Neu5Ac(a2-6)]GlcNAc(b1-2)Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc", dialect="simple")
dt, ax = plot.plot(glycan_glypy, label=True)
ax.figure.set_figwidth(8)
ax.figure.set_figheight(4)
lo, hi = ax.get_ylim()
ax.set_ylim(lo / 2, hi / 1.2)

In [ ]:
glycan = parsing_glycan["IUPAC Condensed"][2]
parse_glycan(glycan, state_set, new_pi, new_a_a, new_a_b, new_b)

In [ ]:
print(parsing_glycan["type"][2])
print("Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-2)Man(a1-3)[Man(a1-3)[Man(a1-6)]Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc")
glycan_glypy = iupac.loads("Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-2)Man(a1-3)[Man(a1-3)[Man(a1-6)]Man(a1-6)]Man(b1-4)GlcNAc(b1-4)GlcNAc", dialect="simple")
dt, ax = plot.plot(glycan_glypy, label=True)
ax.figure.set_figwidth(8)
ax.figure.set_figheight(4)
lo, hi = ax.get_ylim()
ax.set_ylim(lo / 2, hi / 1.2)

# 実行結果の説明
各csvファイルとtxtファイルが実行結果になります。

全て対数変換された値で格納されています。

## 名前
*  piは*Initial state probability* π

*  a_aは*State transition probability(parent→child)* α

*  a_bは*State transition probability(elder sibling→younger sibling)* β

*  bは*Label output probability* b

*  timeは学習に費やした時間

*  likelihoodは入力データ全体に対するモデルの尤度

## ファイル名
*  pi_学習した糖鎖データ数_しきい値(ε)_状態数.csv

*  a_a_学習した糖鎖データ数_しきい値(ε)_状態数(parent)_状態数(child).csv

*  a_b_学習した糖鎖データ数_しきい値(ε)_状態数(elder)_状態数(younger).csv

*  b_学習した糖鎖データ数_しきい値(ε)_状態数_ラベル（単糖）の数.csv

*  time_学習した糖鎖データ数_しきい値(ε)_状態数_ラベルの数.txt

*  likelihood_学習した糖鎖データ数_しきい値(ε)_状態数_ラベルの数.csv

## 内容の説明
piは、列番号が状態の名前です。

a_aとa_bは、行番号と列番号が状態の名前です。行番号から列番号へ遷移します（例：0行1列なら状態0から状態1への遷移）。

bはインデックスに状態名が、カラムに単糖名が入っています。


# バグの原因
## パラメータが関数に入れると勝手に変更される問題
Pythonの仕様的に、ミュータブルな変数は「参照渡し」を行う。よって、関数の中でその変数を変更すると全体にも影響してしまう。

[参照渡しと対処法](https://startlab.jp/learning-python/passbyvalue/)

[参照渡しと値渡し](https://www.javadrive.jp/python/userfunc/index3.html)

## main関数
大規模なプログラムではPythonでもmain関数を作成しよう。

そうすることで、グローバル変数になる可能性を0にすることができ、バグが少なくなる（Pythonは裏でC言語が動いている）。

[Pythonでmain関数を作成するメリット](https://www.lifewithpython.com/2021/01/python-main-function.html)

[グローバル変数の危険性](https:/https://daeudaeu.com/why-no-global/#i-5/)